# NIBSS Fraud Detection: Model Evaluation and Performance Analysis

This notebook provides comprehensive evaluation of optimized fraud detection models including:
- Statistical significance testing with bootstrap confidence intervals
- ROC and Precision-Recall curve analysis
- Confusion matrix visualization
- McNemar's test for model comparison
- Cross-validation stability analysis

The analysis includes proper statistical testing to ensure robust model performance assessment for the Nigerian banking fraud detection system.

## Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score,
    confusion_matrix, classification_report, precision_recall_curve,
    roc_curve, auc, precision_score, recall_score, accuracy_score
)
from sklearn.utils import resample
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.contingency_tables import mcnemar

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Load data and models
data_splits = joblib.load('../data/processed/data_splits.pkl')
best_models = joblib.load('../models/optimization_results.pkl')['best_models']

X_test = data_splits['X_test']
y_test = data_splits['y_test']

print("Models and test data loaded successfully!")
print(f"Test samples: {len(X_test)}")
print(f"Test fraud rate: {y_test.mean():.4f}")

## Bootstrap Confidence Intervals

We implement bootstrap resampling to calculate robust confidence intervals for our performance metrics, ensuring statistical reliability of our results.

In [ ]:
def bootstrap_metric(y_true, y_pred_proba, y_pred, metric_func, n_bootstrap=1000, confidence=0.95):
    """Calculate bootstrap confidence intervals for a metric"""
    scores = []
    n_samples = len(y_true)

    for _ in range(n_bootstrap):
        # Resample with replacement
        indices = resample(range(n_samples), n_samples=n_samples, random_state=None)

        if metric_func.__name__ in ['roc_auc_score', 'average_precision_score']:
            score = metric_func(y_true.iloc[indices], y_pred_proba[indices])
        else:
            score = metric_func(y_true.iloc[indices], y_pred[indices])

        scores.append(score)

    # Calculate confidence intervals
    alpha = 1 - confidence
    lower = np.percentile(scores, (alpha/2) * 100)
    upper = np.percentile(scores, (1 - alpha/2) * 100)
    mean = np.mean(scores)

    return mean, lower, upper, scores

## Test Set Performance Metrics with Bootstrap Confidence Intervals

Calculate comprehensive performance metrics with 95% bootstrap confidence intervals for all models.

In [ ]:
# Initialize results storage
test_metrics = {}

# Generate predictions and probabilities for the test set
predictions = {}
probabilities = {}

for model_name, model in best_models.items():
    print(f"Generating predictions for {model_name} on test set...")
    probabilities[model_name] = model.predict_proba(X_test)[:, 1]
    predictions[model_name] = model.predict(X_test)
    print(f"Predictions generated for {model_name}")

# Metrics to calculate
metrics = {
    'Precision': precision_score,
    'Recall': recall_score,
    'F1-Score': f1_score,
    'AUC': roc_auc_score,
    'Accuracy': accuracy_score,
    'Specificity': lambda y_true, y_pred: recall_score(1-y_true, 1-y_pred, average='binary')
}

# Calculate metrics for each model
for model_name in best_models.keys():
    model_metrics = {}

    for metric_name, metric_func in metrics.items():
        if metric_name == 'AUC':
            mean, lower, upper, _ = bootstrap_metric(
                y_test, probabilities[model_name], predictions[model_name],
                metric_func, n_bootstrap=1000
            )
        else:
            mean, lower, upper, _ = bootstrap_metric(
                y_test, probabilities[model_name], predictions[model_name],
                metric_func, n_bootstrap=1000
            )

        model_metrics[metric_name] = {
            'mean': mean,
            'lower': lower,
            'upper': upper
        }

    test_metrics[model_name] = model_metrics

print("\nBootstrap confidence intervals calculated")

## Table 4.5: Test Set Performance Metrics with 95% Bootstrap Confidence Intervals

Comprehensive performance summary showing mean performance and confidence intervals for all models.

In [ ]:
table_data = []

for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    row = {'Model': model_name.replace('_', ' ').title()}

    for metric in ['Precision', 'Recall', 'F1-Score', 'AUC', 'Accuracy', 'Specificity']:
        values = test_metrics[model_name][metric]
        row[metric] = f"{values['mean']:.3f}\n[{values['lower']:.3f}, {values['upper']:.3f}]"

    table_data.append(row)

# Create DataFrame
table_4_5 = pd.DataFrame(table_data)

print("\nTable 4.5: Test Set Performance Metrics with 95% Bootstrap Confidence Intervals")
print("="*100)
for idx, row in table_4_5.iterrows():
    print(f"\n{row['Model']}:")
    for col in ['Precision', 'Recall', 'F1-Score', 'AUC', 'Accuracy', 'Specificity']:
        print(f"  {col}: {row[col]}")

# Save table
table_4_5.to_csv('../data/processed/table_4_5.csv', index=False)
print("\nTable 4.5 saved to ../data/processed/table_4_5.csv")

## Optimal Threshold Selection

Find optimal thresholds using Youden's J statistic to maximize sensitivity + specificity - 1.

In [ ]:
def find_optimal_threshold(y_true, y_proba, metric='f1'):
    """Find threshold that maximizes the specified metric"""
    thresholds = np.linspace(0, 1, 1000)
    scores = []

    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)

        if metric == 'f1':
            score = f1_score(y_true, y_pred)
        elif metric == 'youden':
            # Youden's J = Sensitivity + Specificity - 1
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
            sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            score = sensitivity + specificity - 1

        scores.append(score)

    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx], scores[optimal_idx]

# Find optimal thresholds for each model
optimal_thresholds = {}
for model_name in best_models.keys():
    threshold, score = find_optimal_threshold(y_test, probabilities[model_name], metric='youden')
    optimal_thresholds[model_name] = threshold
    print(f"{model_name}: Optimal threshold = {threshold:.3f}")

# Save optimal thresholds
joblib.dump(optimal_thresholds, '../models/optimal_thresholds.pkl')
print("\nOptimal thresholds saved to ../models/optimal_thresholds.pkl")

## Table 4.6: Confusion Matrices at Optimal Thresholds

Detailed confusion matrices showing model performance at optimized decision thresholds.

In [ ]:
# Generate predictions at optimal thresholds
optimal_predictions = {}
for model_name, threshold in optimal_thresholds.items():
    optimal_predictions[model_name] = (probabilities[model_name] >= threshold).astype(int)

# Calculate confusion matrices
confusion_matrices = {}
for model_name in best_models.keys():
    cm = confusion_matrix(y_test, optimal_predictions[model_name])
    confusion_matrices[model_name] = cm

# Create Table 4.6 - Confusion Matrices at Optimal Thresholds
print("\nTable 4.6: Confusion Matrices at Optimal Thresholds")
print("="*70)

for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    cm = confusion_matrices[model_name]
    threshold = optimal_thresholds[model_name]

    print(f"\n{model_name.replace('_', ' ').title()} (Threshold = {threshold:.2f})")
    print("-"*50)
    print(f"{'':20} {'Predicted Negative':20} {'Predicted Positive':20}")
    print(f"{'Actual Negative':20} {cm[0,0]:20,d} {cm[0,1]:20,d}")
    print(f"{'Actual Positive':20} {cm[1,0]:20,d} {cm[1,1]:20,d}")

# Save confusion matrices
table_4_6 = pd.DataFrame({
    'Model': [name.replace('_', ' ').title() for name in confusion_matrices.keys()],
    'Threshold': [optimal_thresholds[name] for name in confusion_matrices.keys()],
    'TN': [cm[0,0] for cm in confusion_matrices.values()],
    'FP': [cm[0,1] for cm in confusion_matrices.values()],
    'FN': [cm[1,0] for cm in confusion_matrices.values()],
    'TP': [cm[1,1] for cm in confusion_matrices.values()]
})
table_4_6.to_csv('../data/processed/table_4_6.csv', index=False)
print("\nTable 4.6 saved to ../data/processed/table_4_6.csv")

## Figure 4.6: ROC Curves for Model Comparison

Receiver Operating Characteristic curves showing discrimination capability across all decision thresholds.

In [ ]:
plt.figure(figsize=(10, 8))

# Plot ROC curves for each model
for model_name, model_label in [('logistic_regression', 'Logistic Regression'),
                                ('random_forest', 'Random Forest'),
                                ('xgboost', 'XGBoost')]:
    fpr, tpr, _ = roc_curve(y_test, probabilities[model_name])
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, linewidth=2,
             label=f'{model_label} (AUC = {roc_auc:.3f})')

# Plot random classifier
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Figure 4.6: ROC Curves for Fraud Detection Models', fontsize=14)
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../docs/images/figure_4_6_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## Figure 4.7: Precision-Recall Curves

Precision-Recall curves showing model performance in the imbalanced dataset context, where precision-recall analysis is often more informative than ROC analysis.

In [ ]:
plt.figure(figsize=(10, 8))

# Plot PR curves for each model
for model_name, model_label in [('logistic_regression', 'Logistic Regression'),
                                ('random_forest', 'Random Forest'),
                                ('xgboost', 'XGBoost')]:
    precision, recall, _ = precision_recall_curve(y_test, probabilities[model_name])
    pr_auc = auc(recall, precision)

    plt.plot(recall, precision, linewidth=2,
             label=f'{model_label} (AUC = {pr_auc:.3f})')

# Plot baseline (random classifier)
baseline = y_test.mean()
plt.plot([0, 1], [baseline, baseline], 'k--', linewidth=1,
         label=f'Random Classifier (AUC = {baseline:.3f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Figure 4.7: Precision-Recall Curves Showing Performance in Imbalanced Dataset', fontsize=14)
plt.legend(loc="upper right", fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../docs/images/figure_4_7_pr_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## Figure 4.7.1: Confusion Matrices Visualization

Visual representation of confusion matrices at optimal thresholds with both raw counts and normalized percentages.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

model_names_display = ['Logistic Regression', 'Random Forest', 'XGBoost']

for idx, (model_name, display_name) in enumerate(zip(['logistic_regression', 'random_forest', 'xgboost'],
                                                     model_names_display)):
    cm = confusion_matrices[model_name]

    # Normalize confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Legitimate', 'Fraud'],
                yticklabels=['Legitimate', 'Fraud'],
                cbar_kws={'label': 'Count'})

    # Add normalized values as text
    for i in range(2):
        for j in range(2):
            axes[idx].text(j + 0.5, i + 0.7, f'({cm_normalized[i, j]:.2%})',
                          ha='center', va='center', fontsize=9, color='red')

    axes[idx].set_title(f'{display_name}\n(Threshold = {optimal_thresholds[model_name]:.3f})', fontsize=12)
    axes[idx].set_xlabel('Predicted', fontsize=11)
    axes[idx].set_ylabel('Actual', fontsize=11)

plt.suptitle('Confusion Matrices Visualization', fontsize=14)
plt.tight_layout()
plt.savefig('../docs/images/figure_4_7_1_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

## McNemar's Test for Statistical Model Comparison

McNemar's test provides statistical significance testing for comparing the performance of different classifiers on the same dataset.

In [ ]:
# Create function for McNemar's test
def perform_mcnemar_test(y_pred1, y_pred2, y_true):
    """Perform McNemar's test between two models"""
    # Create contingency table
    correct1_wrong2 = np.sum((y_pred1 == y_true) & (y_pred2 != y_true))
    wrong1_correct2 = np.sum((y_pred1 != y_true) & (y_pred2 == y_true))

    # McNemar's test
    result = mcnemar([[0, correct1_wrong2], [wrong1_correct2, 0]], exact=False)

    return {
        'b': correct1_wrong2,
        'c': wrong1_correct2,
        'statistic': result.statistic,
        'p_value': result.pvalue
    }

# Perform pairwise comparisons
comparisons = [
    ('logistic_regression', 'random_forest', 'Logistic vs Random Forest'),
    ('logistic_regression', 'xgboost', 'Logistic vs XGBoost'),
    ('random_forest', 'xgboost', 'Random Forest vs XGBoost')
]

mcnemar_results = []

for model1, model2, comparison_name in comparisons:
    result = perform_mcnemar_test(
        optimal_predictions[model1],
        optimal_predictions[model2],
        y_test
    )

    # Determine conclusion
    if result['p_value'] < 0.001:
        # Determine which model is better
        if result['c'] > result['b']:
            conclusion = f"{model2.replace('_', ' ').title()} significantly better"
        else:
            conclusion = f"{model1.replace('_', ' ').title()} significantly better"
    else:
        conclusion = "No significant difference"

    mcnemar_results.append({
        'Model Comparison': comparison_name,
        'b (Model 1 wrong, Model 2 correct)': result['b'],
        'c (Model 1 correct, Model 2 wrong)': result['c'],
        'χ² Statistic': f"{result['statistic']:.2f}",
        'p-value': '< 0.001' if result['p_value'] < 0.001 else f"{result['p_value']:.4f}",
        'Conclusion': conclusion
    })

# Create Table 4.7
table_4_7 = pd.DataFrame(mcnemar_results)

print("\nTable 4.7: McNemar's Test Results")
print("="*100)
print(table_4_7.to_string(index=False))

# Save table
table_4_7.to_csv('../data/processed/table_4_7.csv', index=False)
print("\nTable 4.7 saved to ../data/processed/table_4_7.csv")

## Cross-Validation Stability Analysis

Analysis of model stability across cross-validation folds to assess consistency of performance.

In [ ]:
# Load CV results from optimization
optimization_results = joblib.load('../models/optimization_results.pkl')
cv_results = optimization_results['cv_results']
primary_metric = 'roc_auc'  # Define the primary metric

stability_data = []

for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    # Get best model CV results
    model_cv = cv_results[model_name]
    best_idx = model_cv[f'rank_test_{primary_metric}'] == 1

    # Extract AUC scores for each fold
    auc_scores = [model_cv[f'split{i}_test_auc_roc'][best_idx].values[0] for i in range(3)]
    f1_scores = [model_cv[f'split{i}_test_f1'][best_idx].values[0] for i in range(3)]

    # Calculate statistics
    auc_mean = np.mean(auc_scores)
    auc_std = np.std(auc_scores)
    auc_cv = (auc_std / auc_mean) * 100  # Coefficient of variation

    f1_mean = np.mean(f1_scores)
    f1_std = np.std(f1_scores)
    f1_cv = (f1_std / f1_mean) * 100

    stability_data.append({
        'Model': model_name.replace('_', ' ').title(),
        'AUC Mean ± SD': f"{auc_mean:.3f} ± {auc_std:.3f}",
        'AUC CV*': f"{auc_cv:.2f}%",
        'F1-Score Mean ± SD': f"{f1_mean:.3f} ± {f1_std:.3f}",
        'F1 CV*': f"{f1_cv:.2f}%",
        'Stability Rank**': 0  # Will calculate after
    })

# Rank by average CV (lower is better)
stability_df = pd.DataFrame(stability_data)
stability_df['avg_cv'] = stability_df.apply(lambda x:
    (float(x['AUC CV*'].replace('%', '')) + float(x['F1 CV*'].replace('%', ''))) / 2, axis=1)
stability_df = stability_df.sort_values('avg_cv')
stability_df['Stability Rank**'] = range(1, len(stability_df) + 1)
stability_df = stability_df.drop('avg_cv', axis=1)

# Create Table 4.8
print("\nTable 4.8: Cross-Validation Stability Analysis (3 CV Folds)")
print("="*80)
print(stability_df.to_string(index=False))
print("\n*CV = Coefficient of Variation (lower is better)")
print("**Stability Rank: 1 = most stable, 3 = least stable")

# Save table
stability_df.to_csv('../data/processed/table_4_8.csv', index=False)
print("\nTable 4.8 saved to ../data/processed/table_4_8.csv")

## Figure 4.8: AUC Scores with Bootstrap Confidence Intervals

Violin plot visualization showing the distribution of bootstrap AUC scores with confidence intervals.

In [ ]:
# Get bootstrap distributions
bootstrap_aucs = {}
for model_name in best_models.keys():
    _, _, _, scores = bootstrap_metric(
        y_test, probabilities[model_name], predictions[model_name],
        roc_auc_score, n_bootstrap=1000
    )
    bootstrap_aucs[model_name] = scores

# Create violin plot
fig, ax = plt.subplots(figsize=(10, 8))

positions = [1, 2, 3]
model_names_display = ['Logistic\nRegression', 'Random\nForest', 'XGBoost']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

# Create violin plots
parts = ax.violinplot(
    [bootstrap_aucs['logistic_regression'],
     bootstrap_aucs['random_forest'],
     bootstrap_aucs['xgboost']],
    positions=positions,
    showmeans=True,
    showextrema=True,
    widths=0.7
)

# Customize violin plots
for i, pc in enumerate(parts['bodies']):
    pc.set_facecolor(colors[i])
    pc.set_alpha(0.7)
    pc.set_edgecolor('black')
    pc.set_linewidth(1)

# Customize other elements
parts['cmeans'].set_color('black')
parts['cmeans'].set_linewidth(2)
parts['cbars'].set_color('black')
parts['cmaxes'].set_color('black')
parts['cmins'].set_color('black')

# Add confidence interval boxes
for i, model_name in enumerate(['logistic_regression', 'random_forest', 'xgboost']):
    mean = test_metrics[model_name]['AUC']['mean']
    lower = test_metrics[model_name]['AUC']['lower']
    upper = test_metrics[model_name]['AUC']['upper']

    # Draw confidence interval as a box
    box_width = 0.15
    rect = plt.Rectangle((positions[i] - box_width/2, lower),
                        box_width, upper - lower,
                        facecolor='white',
                        edgecolor='black',
                        linewidth=2,
                        zorder=10)
    ax.add_patch(rect)

    # Add mean line
    ax.plot([positions[i] - box_width/2, positions[i] + box_width/2],
            [mean, mean], 'k-', linewidth=3, zorder=11)

    # Add text annotations
    ax.text(positions[i], ax.get_ylim()[1] * 0.98, f'{mean:.4f}',
            ha='center', va='top', fontsize=11, fontweight='bold')
    ax.text(positions[i], lower - 0.001, f'[{lower:.4f}, {upper:.4f}]',
            ha='center', va='top', fontsize=9, rotation=0)

# Calculate appropriate y-axis limits
all_scores = np.concatenate(list(bootstrap_aucs.values()))
y_min = min(all_scores.min(), min([test_metrics[m]['AUC']['lower'] for m in best_models.keys()])) - 0.005
y_max = max(all_scores.max(), max([test_metrics[m]['AUC']['upper'] for m in best_models.keys()])) + 0.005

ax.set_ylim([y_min, y_max])
ax.set_xlim([0.5, 3.5])

ax.set_xticks(positions)
ax.set_xticklabels(model_names_display, fontsize=12)
ax.set_ylabel('AUC-ROC Score', fontsize=13)
ax.set_title('Figure 4.8: AUC Scores with 95% Bootstrap Confidence Intervals', fontsize=15, pad=20)
ax.grid(True, axis='y', alpha=0.3, linestyle='--')

# Add a legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=colors[0], alpha=0.7, label='Logistic Regression'),
    Patch(facecolor=colors[1], alpha=0.7, label='Random Forest'),
    Patch(facecolor=colors[2], alpha=0.7, label='XGBoost'),
    Patch(facecolor='white', edgecolor='black', linewidth=2, label='95% CI')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=10)

plt.tight_layout()
plt.savefig('../docs/images/figure_4_8_auc_bootstrap.png', dpi=300, bbox_inches='tight')
plt.show()

# Print summary statistics
print("\nBootstrap AUC Statistics:")
print("="*50)
for model_name in ['logistic_regression', 'random_forest', 'xgboost']:
    scores = bootstrap_aucs[model_name]
    print(f"\n{model_name.replace('_', ' ').title()}:")
    print(f"  Mean: {np.mean(scores):.4f}")
    print(f"  Std: {np.std(scores):.4f}")
    print(f"  Min: {np.min(scores):.4f}")
    print(f"  Max: {np.max(scores):.4f}")
    print(f"  95% CI: [{test_metrics[model_name]['AUC']['lower']:.4f}, "
          f"{test_metrics[model_name]['AUC']['upper']:.4f}]")

## Save Results

Compile and save all evaluation results for subsequent analysis.

In [ ]:
# Compile all results
evaluation_results = {
    'predictions': predictions,
    'probabilities': probabilities,
    'optimal_predictions': optimal_predictions,
    'optimal_thresholds': optimal_thresholds,
    'test_metrics': test_metrics,
    'confusion_matrices': confusion_matrices,
    'mcnemar_results': mcnemar_results,
    'bootstrap_aucs': bootstrap_aucs,
    'tables': {
        'table_4_5': table_4_5,
        'table_4_6': table_4_6,
        'table_4_7': table_4_7,
        'table_4_8': stability_df
    }
}

joblib.dump(evaluation_results, '../models/evaluation_results.pkl')

print("All evaluation results saved successfully!")
print("\nFiles created:")
print("- ../models/evaluation_results.pkl")
print("- ../models/optimal_thresholds.pkl")
print("- ../data/processed/table_4_5.csv")
print("- ../data/processed/table_4_6.csv")
print("- ../data/processed/table_4_7.csv")
print("- ../data/processed/table_4_8.csv")
print("- ../docs/images/figure_4_6_roc_curves.png")
print("- ../docs/images/figure_4_7_pr_curves.png")
print("- ../docs/images/figure_4_7_1_confusion_matrices.png")
print("- ../docs/images/figure_4_8_auc_bootstrap.png")
print("\nReady for feature importance analysis in next notebook!")